In [11]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

# read 데이터
df = pd.read_csv('weatherFinal2.CSV', encoding="cp949")
df.head()
'''
전체 출력문
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)

'''

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


"\n전체 출력문\nwith pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also\n    print(df)\n\n"

In [12]:
df = df[df.IRR != "Y"] # 부정기 없애기 
df = df[df.CNL != "Y"]

# 비행기 취소와 관련된 Column 삭제
df.drop(columns=['CNL', 'CNR'], axis=1, inplace=True)

# 사용되지 않을 것 같은 데이터 일단 삭제
df.drop(columns=['REG', 'IRR'], axis=1, inplace=True)

# 딜레이 이유 (나중에 쓰일 듯)
df.drop(columns=['DRR'], axis=1, inplace=True)

############# 날씨 데이터 추가 후 주석 제거할 것.
# 날씨 관련 안쓰는 feature 삭제
df.drop(columns=['rain', 'weatherCode'], axis=1, inplace=True)


KeyError: "['rain' 'weatherCode'] not found in axis"

In [ ]:
# ATT (actual time data)가 널 값인 레코드 삭제
df = df[pd.notnull(df['ATT'])]
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# df['SDT_YY'] = df['SDT_YY'].astype('object')
# df['SDT_MM'] = df['SDT_MM'].astype('object')
# df['SDT_DD'] = df['SDT_DD'].astype('object')

In [ ]:
# ARP와 ODP가 같은 데이터 --> Wrong data => 삭제
df.drop(df[df['ARP'] == df['ODP']].index, inplace=True)


In [ ]:
#--------------- fog 모델 돌리기 --------------

## INPUT : 'temp', 'hum', 'dew', 'windSpeed' 4가지 Column의 DataFrame
from sklearn.preprocessing import MinMaxScaler

def fogPreProcessing (df):
    # 비어있는 값 0으로 대체
    #print(df.isnull().sum())
    df.fillna(0, inplace = True)
  
    # MinMaxScaling
    scaler = MinMaxScaler()
    df[['hum', 'dew','temp','windSpeed']] = scaler.fit_transform(df[['hum', 'dew','temp','windSpeed']])
  
    return df



In [ ]:
df.head()

In [ ]:
fog = df[["temp","hum","dew","windSpeed"]]

fog = fogPreProcessing(fog)

In [ ]:
import pickle
from sklearn.externals import joblib

# 저장된 모델 불러오기
clf_from_joblib = joblib.load('fogmodel.pkl') 

# 저장된 모델로 예측하기
#clf_from_joblib.predict(fog)

In [ ]:
# 예측할 때 확률로가져오기!
result = clf_from_joblib.predict_proba(fog)

fog_column = []

# fog_column에 확률값 저장
for i in result:
    fog_column.append(i[1])
    
print("fog_column :",len(fog_column))

In [ ]:
df['fog'] = fog_column

df.drop(columns=['hum', 'dew','temp','windSpeed'], axis=1, inplace=True)
df.head()
#------------------------------------------------------------

In [ ]:
#------------연결지연 시도--------------------------
df123 = df.copy()
df123['AIR'] = (df123['FLT']).str[1:4]
df123.loc[df123['AOD'] == 'D', 'TEMP'] = df123['ODP'] + '-' + df123['AOD'] + '-' + df123['ATT']
df123.loc[df123['AOD'] == 'A', 'TEMP'] = df123['ARP'] + '-' + df123['AOD'] + '-' + df123['ATT']


In [ ]:

# df123['TEMP2'] = pd.to_datetime(df['ATT'],format= '%H:%M')
# df123 = df123.sort_values(['AIR','FLO','DAY','TEMP2']).reset_index(drop=True)
# df123.drop(['SDT_DY','STT','ATT','ARP','ODP','FLO'], axis = 1, inplace=True)
# df123['ABNORMAL'] = False

# df234 = df123.iloc[0:0]

# # with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
# #     print(df123.head(10000))

# ab = 'A'
# for i, row in df123.iterrows():
#     if row['AOD'] == ab:
#         row.ABNORMAL = True
#     df234.append(row)
#     ab = row['AOD']
    
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(df123.head(100))

# #------------------------------------------------

In [ ]:
# ARP 경로 파생변수 생성
df['ARPODP'] = df['ARP'] + '_' + df['ODP']
df.head()

In [ ]:
from datetime import datetime, date

df['Diff'] = (pd.to_datetime(df['ATT'],format= '%H:%M') - pd.to_datetime(df['STT'],format= '%H:%M')).dt.seconds.astype('int64')

# STT와 ATT 격차 큰 순대로 정렬
df = df.sort_values(by=['Diff'], ascending=False)

########################################################################출발
# 딜레이가 최대 5시간이라고 가정했을 때, --> 즉, 2시간 초과한 딜레이는 wrong값이라 가정
max_delay_hour = 5
max_delay = max_delay_hour * 3600 # seconds

# 출발비행기의 경우, 조금이라도 출발이 빠른 건 wrong data라 판단.
# 7200보다 큰 값을 가지는 Diff 데이터 wrong 값 처리
df = df[((df['Diff'] <= max_delay) & (df['AOD']=='D')) | (df['AOD']=='A')]

df.head(100)

########################################################################도착
#이정도는 늦게 도착해도 O
#2시간은 예상보다 늦게도착할 수 있다. 그 이상은 말이안된다
max_delay_hour_arr = 5
max_delay_arr = max_delay_hour_arr * 3600 # seconds

#몇분 일찍도착해도 O
#30분은 예상보다 빨리도착할 수 있음. 그거보다 빨리도착하는 건 말이 안됨
min_delay = 30*60
min_delay = 86400 - min_delay  # 86400(24시간)보다 위인거만 살려놓기
df = df[(df['AOD']=='D') |((df['Diff'] <= max_delay_arr) & (df['AOD']=='A')) | ((df['AOD']=='A')& (df['Diff'] >= min_delay )) ]
df.loc[df['Diff'] >=min_delay, 'Diff'] = df.loc[df['Diff'] >=min_delay, 'Diff']  - 86400
df.head(100)

In [ ]:
# # Check the numerical data
# numerical_feature = [col for col in df.columns if df[col].dtypes == 'int64']

# print(numerical_feature)

# def dist_box(df, feature_list):
#     for col in feature_list:
#         plt.figure(figsize=(15, 5))
#         sns.distplot(df.loc[df[col].notnull(), col])
#         plt.title(col)
#         plt.show()
        
#         df[col].plot(kind='box', color='red')
#         plt.show()
        
# print('*'*50)
# print('All')
# dist_box(df, numerical_feature)
# print('*'*50)
# print('Arrive')
# dist_box(df[df['AOD'] == 'A'], numerical_feature)
# print('*'*50)
# print('Departure')
# dist_box(df[df['AOD'] == 'D'], numerical_feature)

In [ ]:
# 시간에서 시 데이터만 추출 ==> 분은 영향을 줄임.
df['ATT'] = pd.to_datetime(df['ATT'],format= '%H:%M').dt.hour
df['STT'] = pd.to_datetime(df['STT'],format= '%H:%M').dt.hour

############ 잠시 삭제
# 일단 FLT는 처리할 방법이 없어서 삭제해놓음
df.drop(['FLT'], axis=1, inplace = True)
#날짜 년, 일 제거
df.drop(['DAY'], axis=1, inplace = True)

df.drop(['ATT','Diff'], axis = 1, inplace=True)


In [ ]:
# Categorical data --> one hot encoding

def one_hot_dummies(df, *args):
    for col in args:
        one_hot_col = pd.get_dummies(df[col])
        new_col = [col + '_' + str(s) for s in one_hot_col.columns]
        one_hot_col.columns = new_col
        df = df.drop([col], axis = 1)
        df = df.join(one_hot_col)d

    return df

df = one_hot_dummies(df, 'ARPODP')



In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Target 레이블링

from sklearn import preprocessing
# DLY도 1과 0으로 데이터 처리
le = preprocessing.LabelEncoder()
df[['DLY']] = le.fit_transform(df[['DLY']])
df[['ARP']] = le.fit_transform(df[['ARP']])
df[['ODP']] = le.fit_transform(df[['ODP']])
df[['FLO']] = le.fit_transform(df[['FLO']])



In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# 도착, 출발 데이터 분리
df_A = df[df['AOD']=='A']
df_D = df[df['AOD']=='D']

# AOD column삭제 
df_A = df_A.drop(['AOD'],axis = 1)
df_D = df_D.drop(['AOD'],axis = 1)

df_A.head(5)

In [ ]:
df.info()

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score #To evaluate our model
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier


In [ ]:
def makeTestSet(df):
    X = df.drop(['DLY'], axis = 1)
    y = df['DLY']
        
    X_tr, X_t, y_tr, y_t = train_test_split(X,y, test_size= 0.3, random_state = 42)
    
    print("X_train set--------------------")
    print("Shape:",X_tr.shape)
    print("Target:")
    print(y_tr.value_counts())
    print()
      
    print("X_test set info-----------------")
    print("Shape:",X_t.shape)
    print("Target:")
    print(y_t.value_counts())
    print()

    return [X_tr, X_t, y_tr, y_t]

X_train_A, X_test_A, y_train_A, y_test_A = makeTestSet(df_A)
X_train_D, X_test_D, y_train_D, y_test_D = makeTestSet(df_D)

In [ ]:
############# 날씨 데이터 추가 후 주석 제거할 것.

# # scaler - weather data
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# df[['cloudTotal', 'dew', 'hpa', 'seeHpa', 'temp', 'visible', 'windSpeed']] = scaler.fit_transform(df[['cloudTotal', 'dew', 'hpa', 'seeHpa', 'temp', 'visible', 'windSpeed']])


In [ ]:
df.isnull().sum()

In [ ]:
print(df[df['visible']==0])


In [ ]:
# --------------변수 중요도 확인하고 상위 OO개 남기기 -------------------

def feature_importance(X_train, y_train, X_test, y_test):
        
    #### Skew Data처리할거면 주석 해제하기!!
    #X_train, y_train = imbalance(X_train, y_train)
    
    
    log_rg = LogisticRegression().fit(X_train, y_train)
    cross_val_score(log_rg, X_train, y_train, cv=5)
    log_rg.score(X_test, y_test)
    print(classification_report(y_test, log_rg.predict(X_test)))

    # X column 개수 출력
    #print(len(X.columns)) 

    # 변수 중요도 
    fi = pd.DataFrame(zip(X_train.columns.values, abs(log_rg.coef_.ravel())))
    fi.columns = ['feature', 'coef']
    fi.sort_values("coef", ascending=False, inplace=True)
    fi = fi.reset_index().drop(['index'], axis=1)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(fi)

    for index, val in enumerate(fi.iloc[:, 1]):
        if val < 0.1:
            X_train.drop([fi.iloc[index, 0]], axis = 1, inplace = True)
            X_test.drop([fi.iloc[index, 0]], axis = 1, inplace = True)   
    return

feature_importance(X_train_A, y_train_A, X_test_A, y_test_A)
feature_importance(X_train_D, y_train_D, X_test_D, y_test_D)
print(len(X_train_A.columns))
print(len(X_test_A.columns))
print(len(X_train_D.columns))
print(len(X_test_D.columns))

#--------------------------------------------------------------------

In [ ]:
def imbalance (X_train, y_train):

    # 모델설정
    sm = SMOTE(ratio='auto', kind='regular')

    # train데이터를 넣어 복제함
    X_resampled, y_resampled = sm.fit_sample(X_train,list(y_train))

    print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
    print('After OverSampling, the shape of train_y: {} \n'.format(X_resampled.shape))

    print("After OverSampling, counts of label '1': {}".format(sum(y_resampled==1)))
    print("After OverSampling, counts of label '0': {}".format(sum(y_resampled==0)))
    
    return [X_resampled, y_resampled]

In [ ]:
# Under sampling 부분 --> 사용시 runModel의 주석을 풀어 사용해야함.

from imblearn.under_sampling import *
def under_sampling(X_train, y_train):
    X_resampled, y_resampled = RandomUnderSampler(random_state=0).fit_sample(X_train, y_train)
    X_resampled = pd.DataFrame(X_resampled)
    X_resampled.columns = X_train_D.columns
    y_resampled = pd.DataFrame(y_resampled)

    return X_resampled, y_resampled

In [ ]:
from imblearn.under_sampling import ClusterCentroids # doctest: +NORMALIZE_WHITESPACE
from imblearn.under_sampling import EditedNearestNeighbours # doctest: +NORMALIZE_WHITESPACE

def another_under_sampling(X_train, y_train):
    sampler = EditedNearestNeighbours(ratio={0:40000, 1:20000})
    X_train, y_train = sampler.fit_resample(X_train, y_train)

In [ ]:
# ROC Curve그리기
def plot_roc_curve(fpr, tpr):
    
    
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [ ]:
def confusion_matrix_heatmap(y_true, y_pred):
    data = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(data, columns=np.unique(y_true), index = np.unique(y_true))
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (10,7))
    sns.set(font_scale=1.4)#for label size
    sns.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16})# font size

In [ ]:
# grid Search를 통한 하이퍼 파라미터 최적화 부분 --> 아직 적용하지 않음.
# Colab을 통해 적용에 문제가 없나 실험중에 있음. 

def optimizationPrams():
    param_grid = { 
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
    }
    rfc=RandomForestClassifier(random_state=42)

    CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)

    return CV_rfc

In [ ]:
def runModel (X_train, y_train, X_test, y_test, base = True):
    
    #X_train, y_train = under_sampling(X_train, y_train)
    
     ## Skew Data처리할거면 주석 풀기!!
    X_train, y_train = imbalance(X_train, y_train)
    
    models = []
    
    if base ==True:
         models.append(('XGB', XGBClassifier()))
    else:
#         models.append(('LR', LogisticRegression()))
#         models.append(('LDA', LinearDiscriminantAnalysis()))
#         models.append(('KNN', KNeighborsClassifier()))
#         models.append(('CART', DecisionTreeClassifier()))
#         models.append(('NB', GaussianNB()))
        models.append(('RF', RandomForestClassifier(max_depth=4, n_estimators=100, random_state=0)))
#         models.append(('SVM', SVC(gamma='auto')))
#         models.append(('XGB', XGBClassifier()))

    
    # 평가
    results = []
    names = []
    scoring = 'recall'

    seed = 7
    for name, model in models:
        # K-Fold
#         kfold = KFold(n_splits=10, random_state=seed)
#         cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
#         results.append(cv_results)
#         names.append(name)
#         msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
#         print(msg)

        # Hold out 
        model.fit(X_train, y_train)
        y_predict = pd.Series(model.predict(X_test))

        # Resets index to compare original test data with predicted data
        y_test = y_test.reset_index(drop=True)
        y_predict = y_predict.reset_index(drop=True)

#         plt.figure(figsize=(15, 5))
#         plt.scatter(range(y_test.shape[0]), y_test, c='gray')
#         plt.scatter(range(y_predict.shape[0]), y_predict , c='r')
#         diff = abs(y_test - y_predict)
#         plt.bar(range(diff.shape[0]), diff, color='gray')
#         plt.title('Result - Original comparsion')
#         plt.legend(['Original', 'Predict'])
#         plt.show()

        print(model.score(X_test, y_test))
        print('-' * 50)
        
        #--------ROC Curve-----------------
        probs = model.predict_proba(X_test)
        probs = probs[:, 1]
        auc = roc_auc_score(y_test, probs)
        print('AUC: %.2f' % auc)
        
        fpr, tpr, thresholds = roc_curve(y_test, probs)
        plot_roc_curve(fpr, tpr)
        #-----------------------------------
        
        #-------- Confusion matrix heatmap -----------------
        confusion_matrix_heatmap(y_test, y_predict)
        print(classification_report(y_test, y_predict))
        #-----------------------------------
        
             
#      # boxplot algorithm comparison
#     fig = plt.figure(figsize=(11,6))
#     fig.suptitle('Algorithm Comparison')
#     ax = fig.add_subplot(111)
#     plt.boxplot(results)
#     ax.set_xticklabels(names)
#     plt.show()
    
    return

In [ ]:
# True면 base algorithm만 실행하겠다 (base algorithm : Random Forest)
# False면 모든 알고리즘을 실행하겠다.
runModel (X_train_D,y_train_D, X_test_D, y_test_D, True)

In [ ]:
# True면 base algorithm만 실행하겠다 (base algorithm : Random Forest)
# False면 모든 알고리즘을 실행하겠다.
runModel (X_train_A,y_train_A, X_test_A, y_test_A, True)